#### **Setup Python**

In [1]:
# Importação - Bibliotecas Python
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import glob
import warnings
import unicodedata
import re
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Remoção de avisos do Python
warnings.filterwarnings("ignore")

#### **Variáveis - Data Preprocessing TOTVS**

In [2]:
# Variável que armazena o caminho (path) das bases de dados presentes na camada raw
path_raw = "../data/raw"

# Variável que armazena o caminho (path) da camada "processed" onde as bases de dados processadas serão armazenadas inicialmente
path_processed = "../data/processed"

#### **Funções - Data Preprocessing TOTVS**

In [3]:
# Função para padronizar strings: maiúsculas, remover acentos e espaços extras
def padronizar_texto(coluna):
    return (
        coluna.astype(str)
              .str.strip()
              .str.upper()
              .apply(lambda x: unicodedata.normalize("NFKD", x)
                      .encode("ascii", errors="ignore").decode("utf-8"))
    )

def padronizar_valor(coluna):
    return (
        coluna.astype(str)
            .replace("[R$ ]", "", regex=True)
            .str.replace(".", "", regex=False)
            .str.replace(",", ".", regex=False)
            .astype(float)
            .round(2)
    )

# Função para calcular a idade em meses
def idade_contrato_meses(data_assinatura) -> float | int:
    if pd.isnull(data_assinatura):
        return np.nan
    delta = relativedelta(pd.Timestamp.today(), data_assinatura)
    return delta.years * 12 + delta.months

# Função que classifica a nota nps
def classificar_nps(nota):
    if nota >= 9:
        return "PROMOTOR"
    elif nota >= 7:
        return "NEUTRO"
    else:
        return "DETRATOR"

def inserir_dataframe_postgres(df, nome_tabela, if_exists='replace', chunksize=100000):
    # Carrega variáveis de ambiente
    load_dotenv(dotenv_path="../.env")

    # Dados de conexão
    usuario = os.getenv('DB_USER')
    senha = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    porta = os.getenv('DB_PORT')
    banco = os.getenv('DB_NAME')

    # Verifica se as variáveis estão preenchidas
    if None in [usuario, senha, host, porta, banco]:
        raise ValueError("Alguma variável de ambiente do banco não está definida.")

    # Cria engine de conexão
    conn_string = f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}'
    engine = create_engine(conn_string)

    # Envia o DataFrame para o banco
    try:
        df.to_sql(
            name=nome_tabela,
            con=engine,
            if_exists=if_exists,
            index=False,
            chunksize=chunksize,
            method='multi'
        )

        with engine.connect() as conn:
            # Conta número de linhas
            result = conn.execute(text(f"SELECT COUNT(*) FROM {nome_tabela}"))
            total_linhas = result.scalar()

            # Conta número de colunas
            result = conn.execute(text(f"SELECT * FROM {nome_tabela} LIMIT 1"))
            total_colunas = len(result.keys())

        print(f"✅ Tabela '{nome_tabela}' inserida com sucesso!")
        print(f"Total de linhas: {total_linhas:,} | Total de colunas: {total_colunas}\n")

    except Exception as e:
        print(f"❌ Erro ao inserir '{nome_tabela}': {e}")

#### **Extract - Extração Dados Brutos (Raw)**

In [4]:
# Leitura das bases de dados pertencentes há camada "Row"

# Base de dados "dados_clientes.csv"
dados_clientes = pd.read_csv(f"{path_raw}/dados_clientes.csv", sep=";")

# Base de dados "contratacoes_ultimos_12_meses.csv"
contratacoes = pd.read_csv(f"{path_raw}/contratacoes_ultimos_12_meses.csv", sep=";")

# Base de dados "clientes_desde.csv"
clientes_desde = pd.read_csv(f"{path_raw}/clientes_desde.csv", sep=";")

# Base de dados "historico.csv"
historico = pd.read_csv(f"{path_raw}/historico.csv", sep=";")

# Base de dados "mrr.csv"
mrr = pd.read_csv(f"{path_raw}/mrr.csv", sep=";")

# Base de dados "nps.nps_relacional.csv"
nps_relacional = pd.read_csv(f"{path_raw}/nps_relacional.csv", sep=';')

# Base de dados "nps_transacional_aquisicao.csv"
nps_aquisicao = pd.read_csv(f"{path_raw}/nps_transacional_aquisicao.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_implantacao.csv"
nps_implantacao = pd.read_csv(f"{path_raw}/nps_transacional_implantacao.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_onboarding.csv"
nps_onboarding = pd.read_csv(f"{path_raw}/nps_transacional_onboarding.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_produto.csv"
nps_produto = pd.read_csv(f"{path_raw}/nps_transacional_produto.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_suporte.csv"
nps_suporte = pd.read_csv(f"{path_raw}/nps_transacional_suporte.csv", sep=";")

# Ordenação das bases de dados de "telemetria_n.csv"
arquivos_telemetria = sorted(glob.glob(f"{path_raw}/telemetria_*.csv"))

# Base de dados "tickets.csv"
tickets = pd.read_csv(f"{path_raw}/tickets.csv", sep=";")

#### **Transform & Load - Pré-processamento dos dados para a camada "Processed" e PostgreSQL**

##### **Pré-processamento: dados_clientes.csv**

In [118]:
# Pré-processamento da base de dados "dados_clientes.csv"

# Cópia da base mãe
df_dados_clientes = dados_clientes.copy(deep=True)

# Preenchimento de colunas com registros nulos com 'NAO INFORMADO'
colunas_nulo = {
    "DS_SUBSEGMENTO": "NAO INFORMADO",
    "MARCA_TOTVS": "NAO INFORMADO",
    "MODAL_COMERC": "NAO INFORMADO",
    "PERIODICIDADE": "NAO INFORMADO",
    "SITUACAO_CONTRATO": "NAO INFORMADO"
}
for col, valor in colunas_nulo.items():
    df_dados_clientes[col] = df_dados_clientes[col].fillna(valor)

# Padronização em colunas de texto
colunas_texto = [
    "DS_PROD", "DS_LIN_REC", "CIDADE", "DS_CNAE", "DS_SEGMENTO",
    "DS_SUBSEGMENTO", "FAT_FAIXA", "MARCA_TOTVS", "MODAL_COMERC",
    "PERIODICIDADE", "SITUACAO_CONTRATO", "UF"
]
for col in colunas_texto:
    df_dados_clientes[col] = padronizar_texto(df_dados_clientes[col])

# Padronização e criação de faixas de valor para a coluna "VL_TOTAL_CONTRATO"
df_dados_clientes["VL_TOTAL_CONTRATO"] = padronizar_valor(df_dados_clientes["VL_TOTAL_CONTRATO"])
bins = [-float("inf"), 0, 10000, 50000, 100000, 500000, 1000000, float("inf")]
labels = ["ZERADO/NEGATIVO", "ATE 10K", "10K–50K", "50K–100K", "100K–500K", "500K–1M", "ACIMA DE 1M"]
df_dados_clientes["VL_TOTAL_CONTRATO_FAIXA"] = pd.cut(df_dados_clientes["VL_TOTAL_CONTRATO"], bins=bins, labels=labels)

# Conversão da coluna "DT_ASSINATURA_CONTRATO" para datetime
df_dados_clientes["DT_ASSINATURA_CONTRATO"] = pd.to_datetime(df_dados_clientes["DT_ASSINATURA_CONTRATO"], errors="coerce")

# Criação das colunas de ano e mês da assinatura
df_dados_clientes["ANO_ASSINATURA"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].dt.year.astype('int64')
df_dados_clientes["MES_ASSINATURA"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].dt.month.astype('int64')

# Idade do contrato (em meses)
df_dados_clientes["IDADE_CONTRATO_MESES"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].apply(idade_contrato_meses)

# Renomeação de colunas para padrão snake_case
df_dados_clientes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "DS_PROD": "produto",
    "DS_LIN_REC": "linha_receita",
    "CIDADE": "cidade",
    "DS_CNAE": "cnae",
    "DS_SEGMENTO": "segmento",
    "DS_SUBSEGMENTO": "subsegmento",
    "FAT_FAIXA": "faixa_faturamento",
    "MARCA_TOTVS": "marca_totvs",
    "MODAL_COMERC": "modalidade_comercial",
    "PAIS": "pais",
    "PERIODICIDADE": "periodicidade",
    "SITUACAO_CONTRATO": "situacao_contrato",
    "UF": "uf",
    "VL_TOTAL_CONTRATO": "valor_total_contrato",
    "VL_TOTAL_CONTRATO_FAIXA": "faixa_valor_total_contrato",
    "DT_ASSINATURA_CONTRATO": "data_assinatura_contrato",
    "ANO_ASSINATURA": "ano_assinatura",
    "MES_ASSINATURA": "mes_assinatura",
    "IDADE_CONTRATO_MESES": "idade_contrato_meses"
}, inplace=True)

# Reorganização de colunas
df_dados_clientes = df_dados_clientes[[
    "cliente_id", "produto", "linha_receita", "cnae", "segmento", "subsegmento", 
    "faixa_faturamento", "marca_totvs", "modalidade_comercial", "periodicidade",
    "situacao_contrato", "cidade", "uf", "pais", "valor_total_contrato", 
    "faixa_valor_total_contrato", "data_assinatura_contrato", "ano_assinatura", 
    "mes_assinatura", "idade_contrato_meses"
]]

# Exportação da base tratada para a camada "processed"
df_dados_clientes.to_csv(f"{path_processed}/dados_clientes.csv", index=False)

# Carregamento base de dados processada "dados_clientes.csv" no PostreSQL
inserir_dataframe_postgres(df_dados_clientes, "dados_clientes")

✅ Tabela 'dados_clientes' inserida com sucesso!
Total de linhas: 238,597 | Total de colunas: 20



##### **Pré-processamento: contratacoes_ultimos_12_meses.csv**

In [119]:
# Pré-processamento da base de dados "contratacoes_ultimos_12_meses.csv"

# Cópia da base mãe
df_contratacoes = contratacoes.copy(deep=True)

# Padronização de valor para a coluna "VLR_CONTRATACOES_12M"
df_contratacoes["VLR_CONTRATACOES_12M"] = padronizar_valor(df_contratacoes["VLR_CONTRATACOES_12M"])

# Renomeação de colunas para padrão snake_case
df_contratacoes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "QTD_CONTRATACOES_12M": "qtd_contratacoes_12m",
    "VLR_CONTRATACOES_12M": "vlr_contratacoes_12m"
}, inplace=True)

# Reorganização de colunas
df_contratacoes = df_contratacoes[[
    "cliente_id", "qtd_contratacoes_12m", "vlr_contratacoes_12m"
]]

# Exportação da base tratada para a camada "processed"
df_contratacoes.to_csv(f"{path_processed}/contratacoes_ultimos_12_meses.csv", index=False)

# Carregamento base de dados processada "contratacoes_ultimos_12_meses.csv" no PostreSQL
inserir_dataframe_postgres(df_contratacoes, "contratacoes_ultimos_12_meses")

✅ Tabela 'contratacoes_ultimos_12_meses' inserida com sucesso!
Total de linhas: 4,314 | Total de colunas: 3



##### **Pré-processamento: clientes_desde.csv**

In [120]:
# Pré-processamento da base de dados "clientes_desde.csv"

# Cópia da base mãe
df_clientes_desde = clientes_desde.copy(deep=True)

# Renomeação de colunas para padrão snake_case
df_clientes_desde.rename(columns={
    "CLIENTE": "cliente_id",
    "CLIENTE_DESDE": "cliente_desde"
}, inplace=True)

# Conversão da coluna "cliente_desde" para datetime e criação de colunas derivadas
df_clientes_desde["cliente_desde"] = pd.to_datetime(df_clientes_desde["cliente_desde"], errors="coerce")
df_clientes_desde["ano_inicio"] = df_clientes_desde["cliente_desde"].dt.year.astype('int64')
df_clientes_desde["tempo_com_empresa_anos"] = (
    (pd.Timestamp("today") - df_clientes_desde["cliente_desde"]).dt.days / 365
).round(1)

# Reorganização de colunas
df_clientes_desde = df_clientes_desde[[
    "cliente_id", "cliente_desde", "ano_inicio", "tempo_com_empresa_anos"
]]

# Exportação da base tratada para a camada "processed"
df_clientes_desde.to_csv(f"{path_processed}/clientes_desde.csv", index=False)

# Carregamento base de dados processada "clientes_desde.csv" no PostreSQL
inserir_dataframe_postgres(df_clientes_desde, "clientes_desde")

✅ Tabela 'clientes_desde' inserida com sucesso!
Total de linhas: 10,615 | Total de colunas: 4



##### **Pré-processamento: historico.csv**

In [121]:
# Pré-processamento da base de dados "historico.csv"

# Cópia da base mãe
df_historico = historico.copy(deep=True)

# Renomeação de colunas para padrão snake_case
df_historico.rename(columns={
    "NR_PROPOSTA": "nr_proposta",
    "ITEM_PROPOSTA": "item_proposta",
    "DT_UPLOAD": "data_upload",
    "HOSPEDAGEM": "hospedagem",
    "CD_CLI": "cliente_id",
    "FAT_FAIXA": "faixa_faturamento",
    "CD_PROD": "produto_id",
    "QTD": "quantidade",
    "MESES_BONIF": "meses_bonificacao",
    "VL_PCT_DESC_TEMP": "pct_desc_temp",
    "VL_PCT_DESCONTO": "pct_desconto",
    "PRC_UNITARIO": "preco_unitario",
    "VL_DESCONTO_TEMPORARIO": "valor_desc_temp",
    "VL_TOTAL": "valor_total",
    "VL_FULL": "valor_full",
    "VL_DESCONTO": "valor_desconto"
}, inplace=True)

# Padronização de textos
colunas_texto = ["faixa_faturamento", "hospedagem"]
for col in colunas_texto:
    df_historico[col] = padronizar_texto(df_historico[col])

# Conversão da coluna "data_upload" para datetime e criação de colunas derivadas
df_historico["data_upload"] = pd.to_datetime(df_historico["data_upload"], errors="coerce")
df_historico["ano_upload"] = df_historico["data_upload"].dt.year.astype('int64')
df_historico["mes_upload"] = df_historico["data_upload"].dt.month.astype('int64')

# Padronização e conversão de valores numéricos
colunas_valores = [
    "quantidade", "pct_desc_temp", "pct_desconto", "preco_unitario",
    "valor_desc_temp", "valor_total", "valor_full", "valor_desconto"
]
for col in colunas_valores:
    df_historico[col] = padronizar_valor(df_historico[col])

# Reorganização de colunas
df_historico = df_historico[[
    "nr_proposta", "item_proposta", "cliente_id", "produto_id", "data_upload", "ano_upload", "mes_upload",
    "hospedagem", "faixa_faturamento", "quantidade", "meses_bonificacao", "pct_desc_temp", "pct_desconto", 
    "preco_unitario", "valor_desc_temp", "valor_total", "valor_full", "valor_desconto"
]]

# Exportação da base tratada para a camada "processed"
df_historico.to_csv(f"{path_processed}/historico.csv", index=False)

# Carregamento base de dados processada "historico.csv" no PostreSQL
inserir_dataframe_postgres(df_historico, "historico")

✅ Tabela 'historico' inserida com sucesso!
Total de linhas: 22,740 | Total de colunas: 18



##### **Pré-processamento: mrr.csv**

In [122]:
# Pré-processamento da base de dados "mrr.csv"

# Cópia da base mãe
df_mrr = mrr.copy(deep=True)

# Renomeação de colunas para padrão snake_case
df_mrr.rename(columns={
    "CLIENTE": "cliente_id",
    "MRR_12M": "mrr_12m"
}, inplace=True)

# Arredondar MRR para 2 casas decimais e criação de faixas de MRR
df_mrr["mrr_12m"] = df_mrr["mrr_12m"].round(2)
bins = [-0.01, 0, 1000, 5000, 10000, 50000, 100000, float("inf")]
labels = ["ZERADO", "ATE 1K", "1K–5K", "5K–10K", "10K–50K", "50K–100K", "ACIMA DE 100K"]
df_mrr["faixa_mrr_12m"] = pd.cut(df_mrr["mrr_12m"], bins=bins, labels=labels)

# Reogranização de colunas
df_mrr = df_mrr[[
    "cliente_id", "mrr_12m", "faixa_mrr_12m"
]]

# Exportação da base tratada para a camada "processed"
df_mrr.to_csv(f"{path_processed}/mrr.csv", index=False)

# Carregamento base de dados processada "mrr.csv" no PostreSQL
inserir_dataframe_postgres(df_mrr, "mrr")

✅ Tabela 'mrr' inserida com sucesso!
Total de linhas: 7,309 | Total de colunas: 3



##### **Pré-processamento: nps_relacional.csv**

In [123]:
# Pré-processamento da base de dados "nps_relacional.csv"

# Cópia da base mãe
df_nps_relacional = nps_relacional.copy(deep=True)

# Renomeação de colunas para padrão snake_case
df_nps_relacional.rename(columns={
    "respondedAt": "data_resposta",
    "metadata_codcliente": "cliente_id",
    "resposta_NPS": "nota_nps",
    "resposta_unidade": "nota_unidade",
    "Nota_SupTec_Agilidade": "nota_suptec_agilidade",
    "Nota_SupTec_Atendimento": "nota_suptec_atendimento",
    "Nota_Comercial": "nota_comercial",
    "Nota_Custos": "nota_custos",
    "Nota_AdmFin_Atendimento": "nota_admfin_atendimento",
    "Nota_Software": "nota_software",
    "Nota_Software_Atualizacao": "nota_software_atualizacao"
}, inplace=True)

# Conversão da coluna "data_resposta" para datetime e conversão de tipos
df_nps_relacional["data_resposta"] = pd.to_datetime(df_nps_relacional["data_resposta"], errors="coerce")

# Conversão da coluna "cliente_id" para string
df_nps_relacional["cliente_id"] = df_nps_relacional["cliente_id"].astype(str)

# Conversão da coluna "nota_nps" para int64
df_nps_relacional["nota_nps"] = df_nps_relacional["nota_nps"].astype('int64')

# Aplicação da função de classificação da nota NPS
df_nps_relacional["categoria_nps"] = df_nps_relacional["nota_nps"].apply(classificar_nps)

# Padronização e conversão de demais colunas de nota para float
colunas_nota = [
    "nota_unidade", "nota_suptec_agilidade", "nota_suptec_atendimento", "nota_comercial", 
    "nota_custos", "nota_admfin_atendimento", "nota_software", "nota_software_atualizacao"]
df_nps_relacional[colunas_nota] = df_nps_relacional[colunas_nota].astype(float)

# Reorganização de colunas
df_nps_relacional = df_nps_relacional[[
    "cliente_id", "data_resposta", "nota_nps", "nota_unidade", 
    "nota_suptec_agilidade", "nota_suptec_atendimento", 
    "nota_comercial", "nota_custos", "nota_admfin_atendimento", 
    "nota_software", "nota_software_atualizacao", "categoria_nps"
]]

# Exportação da base tratada para a camada "processed"
df_nps_relacional.to_csv(f"{path_processed}/nps_relacional.csv", index=False)

# Carregamento base de dados processada "nps_relacional.csv" no PostreSQL
inserir_dataframe_postgres(df_nps_relacional, "nps_relacional")

✅ Tabela 'nps_relacional' inserida com sucesso!
Total de linhas: 14,143 | Total de colunas: 12



##### **Pré-processamento: nps_transacional_aquisicao.csv**

In [124]:
# Pré-processamento da base de dados "nps_transacional_aquisicao.csv"

# Cópia da base mãe
df_nps_aquisicao = nps_aquisicao.copy(deep=True)

# Renomear colunas para padrão snake_case
df_nps_aquisicao.rename(columns={
    "Cód. Cliente": "cliente_id",
    "Data da Resposta": "data_resposta",
    "Nota NPS": "nota_nps",
    "Nota Agilidade": "nota_agilidade",
    "Nota Conhecimento": "nota_conhecimento",
    "Nota Custo": "nota_custo",
    "Nota Facilidade": "nota_facilidade",
    "Nota Flexibilidade": "nota_flexibilidade"
}, inplace=True)

# Preenchendo valores nulos da coluna "cliente_id" e transformando para string
df_nps_aquisicao["cliente_id"] = df_nps_aquisicao["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Conversão da coluna "data_resposta" para datetime
df_nps_aquisicao["data_resposta"] = pd.to_datetime(df_nps_aquisicao["data_resposta"], errors="coerce")

# Conversão da coluna "nota_nps" para int64
df_nps_aquisicao["nota_nps"] = df_nps_aquisicao["nota_nps"].astype('int64')

# Aplicação da função de classificação da nota NPS
df_nps_aquisicao["categoria_nps"] = df_nps_aquisicao["nota_nps"].apply(classificar_nps)

# Padronização e conversão de demais colunas de nota para float
colunas_nota = ["nota_agilidade", "nota_conhecimento", "nota_custo", "nota_facilidade", "nota_flexibilidade"]
df_nps_aquisicao[colunas_nota] = df_nps_aquisicao[colunas_nota].astype(float)

# Exportação da base tratada para a camada "processed"
df_nps_aquisicao.to_csv(f"{path_processed}/nps_transacional_aquisicao.csv", index=False)

# Carregamento base de dados processada "nps_transacional_aquisicao.csv" no PostreSQL
inserir_dataframe_postgres(df_nps_aquisicao, "nps_transacional_aquisicao")

✅ Tabela 'nps_transacional_aquisicao' inserida com sucesso!
Total de linhas: 178 | Total de colunas: 9



##### **Pré-processamento: nps_transacional_implantacao.csv**

In [125]:
# Pré-processamento da base de dados "nps_transacional_implantacao.csv"

# Cópia da base mãe
df_nps_implatacao = nps_implantacao.copy(deep=True)

# Renomear colunas para padrão snake_case
df_nps_implatacao.rename(columns={
    "Cód. Cliente": "cliente_id",
    "Data da Resposta": "data_resposta",
    "Nota NPS": "nota_nps",
    "Nota Metodologia": "nota_metodologia",
    "Nota Gestao": "nota_gestao",
    "Nota Conhecimento": "nota_conhecimento",
    "Nota Qualidade": "nota_qualidade",
    "Nota Comunicacao": "nota_comunicacao",
    "Nota Prazos": "nota_prazos"
}, inplace=True)

# Preenchendo valores nulos da coluna "cliente_id" e transformando para string
df_nps_implatacao["cliente_id"] = df_nps_implatacao["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Conversão da coluna "data_resposta" para datetime
df_nps_implatacao["data_resposta"] = pd.to_datetime(df_nps_implatacao["data_resposta"], errors="coerce")

# Conversão da coluna "nota_nps" para int64
df_nps_implatacao["nota_nps"] = df_nps_implatacao["nota_nps"].astype('int64')

# Aplicação da função de classificação da nota NPS
df_nps_implatacao["categoria_nps"] = df_nps_implatacao["nota_nps"].apply(classificar_nps)

# Padronização e conversão de demais colunas de nota para float
colunas_nota = ["nota_metodologia", "nota_gestao", "nota_conhecimento", "nota_qualidade", "nota_comunicacao", "nota_prazos"]
df_nps_implatacao[colunas_nota] = df_nps_implatacao[colunas_nota].astype(float)

# Exportação da base tratada para a camada "processed"
df_nps_implatacao.to_csv(f"{path_processed}/nps_transacional_implantacao.csv", index=False)

# Carregamento base de dados processada "nps_transacional_implantacao.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_implatacao, "nps_transacional_implantacao")

✅ Tabela 'nps_transacional_implantacao' inserida com sucesso!
Total de linhas: 662 | Total de colunas: 10



##### **Pré-processamento: nps_transacional_onboarding.csv**

In [126]:
# Pré-processamento da base de dados "nps_transacional_onboarding.csv"

# Cópia da base mãe
df_nps_onboarding = nps_onboarding.copy(deep=True)

# Renomear colunas para padrão snake_case e nomes mais limpos
df_nps_onboarding.rename(columns={
    "Data de resposta": "data_resposta",
    "Cod Cliente": "cliente_id",
    "Em uma escala de 0 a 10, quanto você recomenda o Onboarding da TOTVS para um amigo ou colega?.": "nota_nps",
    "Em uma escala de 0 a 10, o quanto você acredita que o atendimento CS Onboarding ajudou no início da sua trajetória com a TOTVS?": "nota_ajuda_inicio",
    "- Duração do tempo na realização da reunião de Onboarding;": "nota_tempo_reuniao",
    "- Clareza no acesso aos canais de comunicação da TOTVS;": "nota_clareza_comunicacao",
    "- Clareza nas informações em geral transmitidas pelo CS que lhe atendeu no Onboarding;": "nota_clareza_informacao",
    "- Expectativas atendidas no Onboarding da TOTVS.": "nota_expectativa"
}, inplace=True)

# Preenchendo valores nulos da coluna "cliente_id" e transformando para string
df_nps_onboarding["cliente_id"] = df_nps_onboarding["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Conversão da coluna "data_resposta" para datetime
df_nps_onboarding["data_resposta"] = pd.to_datetime(df_nps_onboarding["data_resposta"], errors="coerce")

# Conversão da coluna "nota_nps" para int64
df_nps_onboarding["nota_nps"] = df_nps_onboarding["nota_nps"].astype('int64')

# Aplicação da função de classificação da nota NPS
df_nps_onboarding["categoria_nps"] = df_nps_onboarding["nota_nps"].apply(classificar_nps)

# Garantir que demais notas sejam float
colunas_nota = ["nota_ajuda_inicio", "nota_tempo_reuniao", "nota_clareza_comunicacao", "nota_clareza_informacao", "nota_expectativa"]
df_nps_onboarding[colunas_nota] = df_nps_onboarding[colunas_nota].astype(float)

# Reorganização de colunas
df_nps_onboarding = df_nps_onboarding[[
    'cliente_id', 'data_resposta', 'nota_nps', 'nota_ajuda_inicio', 'nota_tempo_reuniao', 
    'nota_clareza_comunicacao', 'nota_clareza_informacao', 'nota_expectativa', 'categoria_nps'
]]

# Exportação da base tratada para a camada "processed"
df_nps_onboarding.to_csv(f"{path_processed}/nps_transacional_onboarding.csv", index=False)

# Carregamento base de dados processada "nps_transacional_onboarding.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_onboarding, "nps_transacional_onboarding")

✅ Tabela 'nps_transacional_onboarding' inserida com sucesso!
Total de linhas: 208 | Total de colunas: 9



##### **Pré-processamento: nps_transacional_produto.csv**

In [127]:
# Pré-processamento da base de dados "nps_transacional_produto.csv"

# Cópia da base mãe
df_nps_produto = nps_produto.copy(deep=True)

# Renomear colunas para padrão snake_case e nomes mais limpos
df_nps_produto.rename(columns={
    "Data da Resposta": "data_resposta",
    "Linha de Produto": "linha_produto",
    "Nome do Produto": "nome_produto",
    "Nota": "nota_nps",
    "Cód. T": "cliente_id"
}, inplace=True)

# Organizando colunas
df_nps_produto = df_nps_produto[[
    'cliente_id', 'data_resposta', 'linha_produto', 'nome_produto', 'nota_nps'
]]

# Conversão da coluna "data_resposta" para datetime
df_nps_produto["data_resposta"] = pd.to_datetime(df_nps_produto["data_resposta"], errors="coerce")

# Padronização de texto nas colunas "linha_produto" e "nome_produto"
colunas_texto_produto = ["linha_produto", "nome_produto"]
for col in colunas_texto_produto:
    df_nps_produto[col] = padronizar_texto(df_nps_produto[col])

# Conversão da coluna "cliente_id" para string
df_nps_produto["cliente_id"] = df_nps_produto["cliente_id"].astype(str)

# Conversão da coluna "nota_nps" para int64
df_nps_produto["nota_nps"] = df_nps_produto["nota_nps"].astype('int64')

# Aplicação da função de classificação da nota NPS
df_nps_produto["categoria_nps"] = df_nps_produto["nota_nps"].apply(classificar_nps)

# Exportação da base tratada para a camada "processed"
df_nps_produto.to_csv(f"{path_processed}/nps_transacional_produto.csv", index=False)

# Carregamento base de dados processada "nps_transacional_produto.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_produto, "nps_transacional_produto")

✅ Tabela 'nps_transacional_produto' inserida com sucesso!
Total de linhas: 113,207 | Total de colunas: 6



##### **Pré-processamento: nps_transacional_suporte.csv**

In [128]:
# Pré-processamento da base de dados "nps_transacional_suporte.csv"

# Cópia da base mãe
df_nps_suporte = nps_suporte.copy(deep=True)

# Renomear colunas para padrão snake_case e nomes mais limpos
df_nps_suporte.rename(columns={
    "ticket": "ticket_id",
    "resposta_NPS": "nota_nps",
    "grupo_NPS": "categoria_nps",
    "Nota_ConhecimentoAgente": "nota_conhecimento_agente",
    "Nota_Solucao": "nota_solucao",
    "Nota_TempoRetorno": "nota_tempo_retorno",
    "Nota_Facilidade": "nota_facilidade",
    "Nota_Satisfacao": "nota_satisfacao",
    "cliente": "cliente_id"
}, inplace=True)

# Padronizando texto na coluna "categoria_nps" e substituindo "PASSIVO" por "NEUTRO"
df_nps_suporte["categoria_nps"] = padronizar_texto(df_nps_suporte["categoria_nps"])
df_nps_suporte["categoria_nps"] = df_nps_suporte["categoria_nps"].replace("PASSIVO", "NEUTRO")

# Conversão das colunas "tickets_id" e "nota_nps" para int64
colunas_valores_inteiros = ["ticket_id", "nota_nps"]
for col in colunas_valores_inteiros:
    df_nps_suporte[col] = df_nps_suporte[col].astype('int64')

# Conversão da coluna "cliente_id" para string
df_nps_suporte["cliente_id"] = df_nps_suporte["cliente_id"].astype(str)

# Organizando colunas
df_nps_suporte = df_nps_suporte[[
    "cliente_id", "ticket_id", "nota_nps", "nota_conhecimento_agente", 
    "nota_solucao", "nota_tempo_retorno", "nota_facilidade", "nota_satisfacao", 
    "categoria_nps"
]]

# Exportação da base tratada para a camada "processed"
df_nps_suporte.to_csv(f"{path_processed}/nps_transacional_suporte.csv", index=False)

# Carregamento base de dados processada "nps_transacional_suporte.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_suporte, "nps_transacional_suporte")

✅ Tabela 'nps_transacional_suporte' inserida com sucesso!
Total de linhas: 74,794 | Total de colunas: 9



##### **Pré-processamento: telemetria_1.csv a telemetria_11.csv**

In [ ]:
# Pré-processamento das bases de dados "telemetria_n.csv"

for arquivo_telemetria in arquivos_telemetria:
    df_telemetria = pd.read_csv(arquivo_telemetria)

    # Conversão da colunas_nota 'referencedatestart' para datetime e criação de colunas derivadas
    df_telemetria['referencedatestart'] = pd.to_datetime(df_telemetria['referencedatestart'], errors='coerce')
    df_telemetria["ano_referencia"] = df_telemetria["referencedatestart"].dt.year.astype('int64')
    df_telemetria["mes_referencia"] = df_telemetria["referencedatestart"].dt.month.astype('int64')

    # Padronização de texto na coluna 'statuslicenca' e preenchenado valores nulos com 'NAO INFORMADO'
    df_telemetria['statuslicenca'] = padronizar_texto(df_telemetria['statuslicenca'])
    df_telemetria['statuslicenca'] = df_telemetria['statuslicenca'].replace('NAN', 'NAO INFORMADO')

    # Conversão da coluna 'eventduration' para float e arredondamento para 2 casas decimais
    df_telemetria['eventduration'] = df_telemetria['eventduration'].astype(float).round(2)

    # Remoção de colunas irrelevantes (tcloud e clienteprime, pois são totalmente nulas)
    df_telemetria.drop(columns=['tcloud', 'clienteprime'], inplace=True)

    # Renomeação de colunas para padrão snake_case
    df_telemetria.rename(columns={
        'clienteid': 'cliente_id',
        'eventduration': 'duracao_evento',
        'moduloid': 'modulo_id',
        'productlineid': 'linha_produto_id',
        'referencedatestart': 'data_referencia',
        'slotid': 'slot_id',
        'statuslicenca': 'status_licenca'
    }, inplace=True)

    # Organizando colunas
    df_telemetria = df_telemetria[[
        'cliente_id', 'data_referencia', 'ano_referencia', 'mes_referencia', 
        'duracao_evento', 'modulo_id', 'linha_produto_id', 'slot_id', 'status_licenca'
    ]]

    # Exportação da base tratada para a camada "processed"
    df_telemetria.to_csv(f"{path_processed}/{arquivo_telemetria.split("\\")[1]}", index=False)
    
    # Carregamento base de dados processada "telemetria_[n].csv" no PostgreSQL
    inserir_dataframe_postgres(df_telemetria, f"telemetria_{re.findall(r'\d+', arquivo_telemetria)[0]}")

✅ Tabela 'telemetria_1' inserida com sucesso!
Total de linhas: 2,535,221 | Total de colunas: 9

✅ Tabela 'telemetria_10' inserida com sucesso!
Total de linhas: 1,884,789 | Total de colunas: 9

✅ Tabela 'telemetria_11' inserida com sucesso!
Total de linhas: 3,485,570 | Total de colunas: 9

✅ Tabela 'telemetria_2' inserida com sucesso!
Total de linhas: 3,298,672 | Total de colunas: 9

✅ Tabela 'telemetria_3' inserida com sucesso!
Total de linhas: 2,047,906 | Total de colunas: 9

✅ Tabela 'telemetria_4' inserida com sucesso!
Total de linhas: 1,574,418 | Total de colunas: 9

✅ Tabela 'telemetria_5' inserida com sucesso!
Total de linhas: 1,681,694 | Total de colunas: 9

✅ Tabela 'telemetria_6' inserida com sucesso!
Total de linhas: 4,939,234 | Total de colunas: 9

✅ Tabela 'telemetria_7' inserida com sucesso!
Total de linhas: 3,017,771 | Total de colunas: 9

✅ Tabela 'telemetria_8' inserida com sucesso!
Total de linhas: 3,234,597 | Total de colunas: 9

✅ Tabela 'telemetria_9' inserida com s

##### **Pré-processamento: tickets.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_suporte.csv"

# Cópia da base mãe
df_tickets = tickets.copy(deep=True)

# Conversão das colunas 'DT_CRIACAO' e 'DT_ATUALIZACAO' para datetime e criação de colunas derivadas
df_tickets['DT_CRIACAO'] = pd.to_datetime(df_tickets['DT_CRIACAO'], errors='coerce')
df_tickets['DT_ATUALIZACAO'] = pd.to_datetime(df_tickets['DT_ATUALIZACAO'], errors='coerce')
df_tickets['ano_criacao'] = df_tickets['DT_CRIACAO'].dt.year.astype('int64')
df_tickets['mes_criacao'] = df_tickets['DT_CRIACAO'].dt.month.astype('int64')

# Padronização de texto nas colunas
colunas_texto = ['NOME_GRUPO', 'TIPO_TICKET', 'STATUS_TICKET', 'PRIORIDADE_TICKET']
for col in colunas_texto:
    df_tickets[col] = padronizar_texto(df_tickets[col])

# Renomeação de colunas para padrão snake_case
df_tickets.rename(columns={
    'CODIGO_ORGANIZACAO': 'cliente_id',
    'NOME_GRUPO': 'nome_grupo',
    'TIPO_TICKET': 'tipo_ticket',
    'STATUS_TICKET': 'status_ticket',
    'DT_CRIACAO': 'data_criacao',
    'DT_ATUALIZACAO': 'data_atualizacao',
    'BK_TICKET': 'ticket_id',
    'PRIORIDADE_TICKET': 'prioridade_ticket'
}, inplace=True)

# Reorganização das colunas
df_tickets = df_tickets[[
    'ticket_id', 'cliente_id', 'nome_grupo', 'tipo_ticket', 
    'status_ticket', 'prioridade_ticket', 'data_criacao', 
    'ano_criacao', 'mes_criacao', 'data_atualizacao'
]]

# Exportação da base tratada para a camada "processed"
df_tickets.to_csv(f"{path_processed}/tickets.csv", index=False)

# Carregamento base de dados processada "tickets.csv" no PostgreSQL
inserir_dataframe_postgres(df_tickets, "tickets")

✅ Tabela 'tickets' inserida com sucesso!
Total de linhas: 976,364 | Total de colunas: 10

